In [7]:
import lzma
import json
import pandas as pd
import seaborn as sns
import os
sns.set_theme()

In [2]:
def load_output(filename):
    with lzma.open(filename, "r") as f:
        return json.load(f)

In [99]:
def scan_events(events):
    extra_data = {"lb_history": [], "ub_history": []}
    for e in events:
        t = e["type"]
        if t == "INITIAL_PHASE_DONE":
            extra_data["initial_phase_lb"] = e["lb"]
            extra_data["initial_phase_ub"] = e["ub"]
            extra_data["initial_phase_time"] = e["time"]
            extra_data["initial_num_interactions"] = e["num_interactions"]
            extra_data["reduced_num_interactions"] = e["num_interactions"]
        elif t == "FIRST_COLORING_DONE":
            extra_data["time_to_first_solution"] = e["time"]
        elif t == "DONE_IMPLIED_VERTEX_ELIMINATION":
            extra_data["reduced_num_interactions"] = e["reduced_size"]
        elif t in {"INITIAL_ITERATION_DONE","STRENGTHENED_CONSTRAINTS",} or "SOLVE_FULL_RELAX" in t:
            continue
        elif t == "LB_IMPROVED" or t == "IMPROVED_LB":
            source = None
            key = None
            if "lb" in e:
                key = "lb"
            else:
                key = "num_interactions"
            if "iteration" in e:
                source = "initial_phase"
            elif 'method' in e:
                source = e['method']
            elif 'source' in e:
                source = e['source']
            if source is None:
                raise RuntimeError("NO SOURCE FOR " + str(e))
            lb = e[key]
            if not extra_data["lb_history"] or extra_data["lb_history"][-1]["lb"] < lb:
                extra_data["lb_history"].append({"time": e["time"], "lb": lb, "source": source})
        elif t == "UB_IMPROVED" or t == "IMPROVED_UB" or t == "IMPROVED_SOLUTION":
            source = None
            key = "num_configs"
            if 'iteration' in e:
                source = "initial_phase"
            elif 'source' in e:
                source = e["source"]
            if key not in e:
                key = "ub"
            if key not in e:
                key = "size"
            if key not in e:
                raise RuntimeError("NO UB FOR "+str(e))
            if source is None:
                raise RuntimeError("NO SOURCE FOR " + str(e))
            extra_data["ub_history"].append({"time": e["time"], "ub": e[key], "source": source})
    return extra_data

In [34]:
data = load_output("02_output/soletta_2017-03-09_21-02-40.out.json.xz")

In [97]:
def load_all_outputs():
    result = []
    for raw_file in os.listdir("02_output"):
        if not raw_file.endswith(".json.xz"):
            continue
        raw_data = load_output(os.path.join("02_output", raw_file))
        extra_data = scan_events(raw_data["events"])
        result.append({
            "instance_name": raw_file[:-12],
            "lb": raw_data["lb"],
            "ub": raw_data["ub"],
            "total_time": raw_data["events"][-1]["time"],
            "mes_size": len(raw_data["mutually_exclusive_set"]),
            **extra_data
        })
    return result

In [100]:
output = load_all_outputs()

In [101]:
pd.DataFrame(output).sort_values("lb")

,instance_name,lb,ub,total_time,mes_size,lb_history,ub_history,time_to_first_solution,initial_phase_lb,initial_phase_ub,initial_phase_time,initial_num_interactions,reduced_num_interactions
13,calculate,5,5,5.126283,4,"[{'time': 0.002787068, 'lb': 2, 'source': 'ini...","[{'time': 0.002143266, 'ub': 5, 'source': 'ini...",0.001661,4,5,5.001665,43,14
49,car,5,5,5.077293,4,"[{'time': 0.002759584, 'lb': 4, 'source': 'ini...","[{'time': 0.002329134, 'ub': 5, 'source': 'ini...",0.001864,4,5,5.001911,24,5
37,lcm,6,6,5.029452,4,"[{'time': 0.002524156, 'lb': 4, 'source': 'ini...","[{'time': 0.001920674, 'ub': 6, 'source': 'ini...",0.001422,4,6,5.001464,42,16
30,email,6,6,5.058378,4,"[{'time': 0.002266883, 'lb': 2, 'source': 'ini...","[{'time': 0.00184709, 'ub': 7, 'source': 'init...",0.001385,4,6,5.001431,64,33
24,ChatClient,7,7,5.340535,5,"[{'time': 0.031295436, 'lb': 2, 'source': 'ini...","[{'time': 0.030846525, 'ub': 11, 'source': 'in...",0.030396,3,7,5.030415,152,113
6,APL,7,7,5.067381,5,"[{'time': 0.002899745, 'lb': 2, 'source': 'ini...","[{'time': 0.002426676, 'ub': 9, 'source': 'ini...",0.001967,3,8,5.001980,273,164
23,FeatureIDE,8,8,16.531017,5,"[{'time': 0.002715204, 'lb': 2, 'source': 'ini...","[{'time': 0.002243892, 'ub': 17, 'source': 'in...",0.001790,3,10,5.001740,489,213
19,toybox_2020-12-06_00-02-46,8,13,3600.017098,7,"[{'time': 0.005607159, 'lb': 3, 'source': 'ini...","[{'time': 0.005055188, 'ub': 20, 'source': 'in...",0.004407,6,17,5.004327,186912,154083
40,FameDB,8,8,5.206051,6,"[{'time': 0.002768445, 'lb': 4, 'source': 'ini...","[{'time': 0.002310386, 'ub': 9, 'source': 'ini...",0.001834,5,9,5.001894,225,118
46,toybox_2006-10-31_23-30-06,8,8,5.058375,8,"[{'time': 0.002758119, 'lb': 3, 'source': 'ini...","[{'time': 0.002092147, 'ub': 13, 'source': 'in...",0.001626,6,10,5.001649,473,111


In [80]:
extra_data = scan_events(data["events"])

COMMAND_LINE_PARSED
INPUT_READ
BEGIN_SIMPLIFICATION
DONE_SIMPLIFICATION
FIRST_SOLVE_BEGINS
FIRST_COLORING_DONE
FEASIBILITIES_EXTRACTED
INFEASIBILITIES_LEARNED
FIRST_SOLVE_DONE
FIRST_LB_BEGINS
FIRST_LB_DONE
BEGIN_IMPLIED_VERTEX_ELIMINATION
DONE_IMPLIED_VERTEX_ELIMINATION
PORTFOLIO_START
LNS_DESTROY_BEGIN
BEGIN_DESTROY_RANDOM_AND_LEAST_UNIQUE
BEGIN_DESTROY_RANDOM_AND_LEAST_UNIQUE_TRIAL
LNS_DESTROY_BEGIN
BEGIN_DESTROY_RANDOM_AND_LEAST_UNIQUE
BEGIN_DESTROY_RANDOM_AND_LEAST_UNIQUE_TRIAL
LNS_DESTROY_BEGIN
LNS_DESTROY_RANDOM_BEGIN_TRIAL
LNS_DESTROY_BEGIN
BEGIN_DESTROY_RANDOM_AND_LEAST_UNIQUE
BEGIN_DESTROY_RANDOM_AND_LEAST_UNIQUE_TRIAL
LNS_DESTROY_BEGIN
LNS_DESTROY_AVOID_DOOM_BEGIN_TRIAL
LNS_DESTROY_AVOID_DOOM_DONE
LNS_DESTROY_BEGIN
LNS_DESTROY_AVOID_DOOM_BEGIN_TRIAL
LNS_DESTROY_AVOID_DOOM_DONE
LNS_DESTROY_BEGIN
BEGIN_DESTROY_RANDOM_AND_LEAST_UNIQUE
BEGIN_DESTROY_RANDOM_AND_LEAST_UNIQUE_TRIAL
LNS_DESTROY_BEGIN
LNS_DESTROY_AVOID_DOOM_BEGIN_TRIAL
LNS_DESTROY_AVOID_DOOM_DONE
LNS_DESTROY_BEGIN
BEG

In [81]:
extra_data

{'lb_history': [{'time': 0.011618832, 'lb': 12, 'source': 'initial_phase'},
  {'time': 0.014612259, 'lb': 15, 'source': 'initial_phase'},
  {'time': 0.01860713, 'lb': 16, 'source': 'initial_phase'},
  {'time': 0.023293395, 'lb': 17, 'source': 'initial_phase'},
  {'time': 0.028527343, 'lb': 18, 'source': 'initial_phase'},
  {'time': 0.03357331, 'lb': 23, 'source': 'initial_phase'},
  {'time': 0.039188791, 'lb': 25, 'source': 'initial_phase'},
  {'time': 0.044465019, 'lb': 26, 'source': 'initial_phase'},
  {'time': 0.054625064, 'lb': 27, 'source': 'initial_phase'},
  {'time': 0.124051766, 'lb': 28, 'source': 'initial_phase'},
  {'time': 0.137981211, 'lb': 29, 'source': 'initial_phase'},
  {'time': 0.217090226, 'lb': 30, 'source': 'initial_phase'},
  {'time': 0.351456418, 'lb': 31, 'source': 'initial_phase'},
  {'time': 5.199997679, 'lb': 32, 'source': 'C&P/SATDSatur Exact Solver'},
  {'time': 5.909142338, 'lb': 33, 'source': 'LP rounding'},
  {'time': 6.465792607, 'lb': 34, 'source': 'C&